# Word Vector
Word2Vec是一种方法，将词语数据化为向量结构。其中包含的方法有多对一的Continuous Bag-of-Word Model和一对多的Skip-Gram，不同的方法训练出来的 词向量所包含掌握的信息也有所不同。\
最本质的思想：一个词的意义完全由它在文本中上下文的词所决定，因此词向量所包含的意义也完全在文本中产生。


**希望它具有的性质**
1. 相似的词的向量接近
2. 关系相似的词对之间的向量关系接近，如（男人，女人）和（国王，女王）
3. 词向量各个维度具有可解释的意义

**过程**\
随机初始化词向量矩阵\
Word2Vec用临近中心词对目标词进行预测。使用softmax函数，将中心词词向量（输入向量）乘上随机初始化的参数矩阵（多个输出向量的stacking）计算词表中所有词出现的概率。反向通过梯度下降让此概率最大

**原理**\
若想使此概率最大化，在梯度下降更新中，实际值对应的输出向量（希望预测概率为1）会加上输入向量的一小部分，而其他希望预测概率为0的则会减去输入向量的一小部分，导致输出向量与输入向量靠近。\
而输入向量在更新时，会减去希望预测概率为0的输出向量的一小部分，加上希望预测概率为1的输出向量的一小部分，因此输入向量向输出向量靠近。\
背后本质的原理是当两向量逐渐接近至完全重合时，计算时点积所得到的值越来越大，反之则越来越小。\
在这个过程中，由于中心词和目标词在不断变化，输入向量和输出向量也在不断互相拉扯，大量训练后逐渐趋于稳定。\
***经过我的思考，这其实正是分类问题梯度下降的本质，在将各参数向量化思考后就是上面的道理。***

**是否具有性质？**
1. 相似的词由于出现的环境相似，即周边词相似，则词向量接近的参数向量相似。
2. 有逻辑关系的两个词一定会在某种语料下同时出现，两词的词向量在这种条件下相互拉扯，最终形成一定的向量夹角，即两个词逻辑关系的向量表示。另外两个具有相似逻辑关系的词会以相似的方式出现在一段语料中，它们词向量的相互拉扯将造就同样的夹角，因此两组逻辑关系的相似性确立。【即使依然看起来玄学，（凭什么共同出现就代表能形成有意义的向量关系），但这是我能想到的最好解释】
3. 既然在建立词向量的初始就是随机生成的，在演化过程中也是相互作用的，从始至终都不是意义导向而是统计数据导向，最终如何能产生各维度可解释的意义？悬吧？

**关于解释与演化**\
词向量的生成过程是几万个词共同演化的结果。像一个多元微分方程，所有变量之间都互相影响，改变一个变量不仅会改变其他变量，还会改变这个变量本身的自变量，类似于形成环路。\
如果一定要思考为什么共同出现就能形成有意义的向量关系，这可能取决于，这两个具有逻辑关系的词周围其他的词，与那两个具有相似逻辑关系的词周围其他的词之间，词与词具有某种相似性，或这些词的逻辑关系之间具有某种相似性，而它们相似性变为向量关系的过程又或许依靠原本这两组词的相似性。这样就产生了循环套娃，这对擅长求解析解的人脑来说难以想象，但对擅长迭代的计算机来说或许是可解的（至少结果上证明词向量的炼金达到了效果）。

# Bias
非但能产生大量逻辑，甚至还能产生过多的逻辑，即偏见。\
Eg. Man: Doctor as Woman: Nurse

消除偏见的方法：人工调参\
1. 首先找到代表了偏见的向量方向（或多个方向，可能会用到SVD分解）。
2. 再将带有偏见的词对投影到与这个方向正交的方向，相当于消除在这一方向上的偏移。
3. 最后把本应当具有偏差的进行平移，均衡分布在正交方向两侧。

但这样是否会破坏词向量本身的构造，即破坏其他维度的含义或其他逻辑关系？我感觉是会的，毕竟这是个自我训练出来的黑箱，突然人手动调试是不可取的。或许有更好的方法。

In [3]:
import torch

In [4]:
print(torch.arange(0,10,2))

tensor([0, 2, 4, 6, 8])
